In [3]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
from transformers import BertTokenizer, BertForMaskedLM
import torch
from transformers import AdamW

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


#BLANC Help reimplementation

In [1]:
class BlancHelp:
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        self.model = BertForMaskedLM.from_pretrained('bert-base-uncased')
        self.M = 6
        self.L_min = 4

    def mask_sentence(self, sentence, i0):
        tokens = self.tokenizer.tokenize(sentence)
        masked_tokens = tokens.copy()
        mask_positions = []

        for i, token in enumerate(tokens):
            if (i - i0) % self.M == 0 and len(token) >= self.L_min:
                masked_tokens[i] = self.tokenizer.mask_token
                mask_positions.append(i)

        masked_sentence = self.tokenizer.convert_tokens_to_string(masked_tokens)

        print(f"Original Sentence: {sentence}")
        print(f"Masked Sentence: {masked_sentence}")
        print(f"Mask Positions: {mask_positions}")

        return masked_sentence, mask_positions

    def unmask(self, text):
        inputs = self.tokenizer(text, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(**inputs)
        predictions = outputs.logits

        predictions = predictions[:, 1:-1, :]

        print(f"Input Text for Unmasking: {text}")
        predicted_tokens = torch.argmax(predictions, dim=-1)
        predicted_sentence = self.tokenizer.decode(predicted_tokens[0])
        print(f"Predicted Sentence: {predicted_sentence}")
        return predictions

    def eval(self, document, summary):
        sentences = sent_tokenize(document)
        S = [[0, 0], [0, 0]]
        B = 0

        for sentence in sentences:
            for i0 in range(1, self.M + 1):
                masked_sentence, mask_positions = self.mask_sentence(sentence, i0)
                input_base = "." * len(summary) + " "+ masked_sentence
                input_help = summary + " " + masked_sentence

                out_base = self.unmask(input_base)
                out_help = self.unmask(input_help)

                for pos in mask_positions:
                    base_pos = pos + len(self.tokenizer.tokenize("." * len(summary)))
                    help_pos = pos + len(self.tokenizer.tokenize(summary))

                    base_pred_token = torch.argmax(out_base[0, base_pos]).item()
                    help_pred_token = torch.argmax(out_help[0, help_pos]).item()
                    original_token = self.tokenizer.convert_tokens_to_ids(self.tokenizer.tokenize(sentence)[pos])

                    k = int(base_pred_token == original_token)
                    m = int(help_pred_token == original_token)

                    base_pred_word = self.tokenizer.convert_ids_to_tokens(base_pred_token)
                    help_pred_word = self.tokenizer.convert_ids_to_tokens(help_pred_token)
                    original_word = self.tokenizer.convert_ids_to_tokens(original_token)

                    print(f"Original Word: {original_word}, Base Prediction: {base_pred_word}, Help Prediction: {help_pred_word}")
                    S[k][m] += 1

        if sum(sum(S, [])) != 0:
            B = (S[0][1] - S[1][0]) / sum(sum(S, []))

        print(f"S: {S}")
        print(f"BLANC Score: {B}")
        return B


## Test code for case study

In [4]:
# Example usage
document = "Jack drove his minivan to the bazaar to purchase milk and honey for his large family."
summary = "Jack bought milk and honey."
blanc_help = BlancHelp()
blanc_score = blanc_help.eval(document, summary)
blanc_score

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Original Sentence: Jack drove his minivan to the bazaar to purchase milk and honey for his large family.
Masked Sentence: jack [MASK] his minivan to the [MASK] to purchase milk and honey for his large family .
Mask Positions: [1, 7]
Input Text for Unmasking: ........................... jack [MASK] his minivan to the [MASK] to purchase milk and honey for his large family .
Predicted Sentence: ........................... jack took his minivan to the store to buy milk and honey for his large family.
Input Text for Unmasking: Jack bought milk and honey. jack [MASK] his minivan to the [MASK] to purchase milk and honey for his large family .
Predicted Sentence: jack bought milk and honey. jack took his minivan to the store to buy milk and honey for his large family.
Original Word: drove, Base Prediction: took, Help Prediction: took
Original Word: bazaar, Base Prediction: store, Help Prediction: store
Original Sentence: Jack drove his minivan to the bazaar to purchase milk and honey for his l

0.1

In [5]:
summary = "Jack drove his car."
blanc_help = BlancHelp()
blanc_score = blanc_help.eval(document, summary)
blanc_score

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Original Sentence: Jack drove his minivan to the bazaar to purchase milk and honey for his large family.
Masked Sentence: jack [MASK] his minivan to the [MASK] to purchase milk and honey for his large family .
Mask Positions: [1, 7]
Input Text for Unmasking: ................... jack [MASK] his minivan to the [MASK] to purchase milk and honey for his large family .
Predicted Sentence: ................... jack took his minivan to the store to buy milk and honey for his large family.
Input Text for Unmasking: Jack drove his car. jack [MASK] his minivan to the [MASK] to purchase milk and honey for his large family .
Predicted Sentence: jack drove his car. jack drove his minivan to the store to buy milk and honey for his large family.
Original Word: drove, Base Prediction: took, Help Prediction: drove
Original Word: bazaar, Base Prediction: store, Help Prediction: store
Original Sentence: Jack drove his minivan to the bazaar to purchase milk and honey for his large family.
Masked Sentence: 

0.2

#BLANC Tune reimplementation

In [12]:
class BlancTune(BlancHelp):
    def __init__(self):
        super().__init__()

    def fine_tune_model(self, summary, learning_rate=5e-5, epochs=3):
        inputs = self.tokenizer(summary, return_tensors="pt", max_length=512, truncation=True, padding="max_length")

        # Prepare the model for training
        self.model.train()
        optimizer = AdamW(self.model.parameters(), lr=learning_rate)

        # Fine-tuning loop
        for epoch in range(epochs):
            optimizer.zero_grad()
            outputs = self.model(**inputs, labels=inputs["input_ids"])
            loss = outputs.loss
            loss.backward()
            optimizer.step()

    def eval(self, document, summary):
        self.fine_tune_model(summary)
        return super().eval(document, summary)


## Test code for case study

In [13]:
blanc_tune = BlancTune()
tune_score = blanc_tune.eval(document, summary)
print(f"BLANC-tune Score: {tune_score}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'bert.pooler.dense.weight', 'bert.pooler.dense.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Original Sentence: Jack drove his minivan to the bazaar to purchase milk and honey for his large family.
Masked Sentence: jack [MASK] his minivan to the [MASK] to purchase milk and honey for his large family .
Mask Positions: [1, 7]
Input Text for Unmasking: ........................... jack [MASK] his minivan to the [MASK] to purchase milk and honey for his large family .
Predicted Sentence: ........................... jack takes his minivan to the store to buy milk and honey for his large family.
Input Text for Unmasking: Jack bought milk and honey. jack [MASK] his minivan to the [MASK] to purchase milk and honey for his large family .
Predicted Sentence: jack bought milk and honey. jack took his mini rover to the store to purchase milk and honey for his large family.
Original Word: drove, Base Prediction: takes, Help Prediction: took
Original Word: bazaar, Base Prediction: store, Help Prediction: store
Original Sentence: Jack drove his minivan to the bazaar to purchase milk and honey